In [1]:
from dash import Dash, html, dcc
import json
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

In [2]:
# Importation of data
path = 'https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7'
df = pd.read_csv(path, sep=';', parse_dates=True, dayfirst=True) 
#deletion of columns 'HospConv', 'SSR_USLD', and 'autres'
df.drop(columns =['HospConv', 'SSR_USLD','autres'], inplace = True)
#loading of the data of the departements of French
deps_path = 'https://www.data.gouv.fr/en/datasets/r/987227fb-dcb2-429e-96af-8979f97c9c84'
deps = pd.read_csv(deps_path)
#Jointure
data = pd.merge(df, deps, left_on = "dep", right_on = "num_dep", how = "left")

In [3]:

#modification of the modalities of the sex variables 
data["sexe"] = data["sexe"].astype(str) # change in sex type
data.loc[data.sexe =='0', "sexe"] = "femme"
data.loc[data.sexe =='1', "sexe"] = "homme"
data.loc[data.sexe =='2', "sexe"] = "autres"
#converting day variable to datetime types
data['jour'] = pd.to_datetime(data['jour'])
# date recovery
data["Annees"] = data["jour"].dt.year
data.head(2)

,dep,sexe,jour,hosp,rea,rad,dc,num_dep,dep_name,region_name,Annees
0,01,femme,2020-03-18,2,0,1,0,01,Ain,Auvergne-Rhône-Alpes,2020
1,01,homme,2020-03-18,1,0,1,0,01,Ain,Auvergne-Rhône-Alpes,2020


,dep,sexe,jour,hosp,rea,rad,dc,num_dep,dep_name,region_name,Annees
0,01,femme,2020-03-18,2,0,1,0,01,Ain,Auvergne-Rhône-Alpes,2020
1,01,homme,2020-03-18,1,0,1,0,01,Ain,Auvergne-Rhône-Alpes,2020


In [4]:
df1 = data.groupby(["Annees", "dep_name", "region_name"], as_index = False)["dc","rea","hosp"].sum()
df1

/var/folders/z5/z205q_gs53g35dpt2nk3cq2w0000gn/T/ipykernel_26021/1580160074.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df1 = data.groupby(["Annees", "dep_name", "region_name"], as_index = False)["dc","rea","hosp"].sum()


,Annees,dep_name,region_name,dc,rea,hosp
0,2020,Ain,Auvergne-Rhône-Alpes,76266,6215,76740
1,2020,Aisne,Hauts-de-France,160520,9463,82775
2,2020,Allier,Auvergne-Rhône-Alpes,41219,4362,41563
3,2020,Alpes-Maritimes,Provence-Alpes-Côte d'Azur,128929,15567,110972
4,2020,Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur,12966,1402,19744
...,...,...,...,...,...,...
399,2023,Vendée,Pays de la Loire,29112,332,2394
400,2023,Vienne,Nouvelle-Aquitaine,27348,392,6126
401,2023,Vosges,Grand Est,61868,382,5334
402,2023,Yonne,Bourgogne-Franche-Comté,51214,398,10464


/var/folders/z5/z205q_gs53g35dpt2nk3cq2w0000gn/T/ipykernel_26021/1580160074.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df1 = data.groupby(["Annees", "dep_name", "region_name"], as_index = False)["dc","rea","hosp"].sum()


,Annees,dep_name,region_name,dc,rea,hosp
0,2020,Ain,Auvergne-Rhône-Alpes,76266,6215,76740
1,2020,Aisne,Hauts-de-France,160520,9463,82775
2,2020,Allier,Auvergne-Rhône-Alpes,41219,4362,41563
3,2020,Alpes-Maritimes,Provence-Alpes-Côte d'Azur,128929,15567,110972
4,2020,Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur,12966,1402,19744
...,...,...,...,...,...,...
399,2023,Vendée,Pays de la Loire,29112,332,2394
400,2023,Vienne,Nouvelle-Aquitaine,27348,392,6126
401,2023,Vosges,Grand Est,61868,382,5334
402,2023,Yonne,Bourgogne-Franche-Comté,51214,398,10464


In [5]:
# functions define


def dataframeRegion(y):
    if (y == "Décés") :
        df = data.groupby("region_name", as_index = False)["dc"].sum()
    elif y == "Réanimation":
        df = data.groupby("region_name", as_index = False)["rea"].sum()
    else :
            df = data.groupby("region_name", as_index = False)["hosp"].sum()
    return df       

    

def dataframeSexe(y):
   
    if (y == "Décés") :
        df = data.groupby("sexe", as_index = False)["dc"].sum()
    elif y == "Réanimation":
        df = data.groupby("sexe", as_index = False)["rea"].sum()
    else :
        df = data.groupby("sexe", as_index = False)["hosp"].sum()
  
    return df
  

In [6]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([dcc.Markdown(''' # DashBoard covid 19 in France''')], style = {"textAlign":"center","animation-timeline": "test_05"}),
    html.Br(),
       html.Div([
            html.Div([
                dcc.Graph(id = "graphe1"),
                dcc.Slider(
                df1["Annees"].min(),
                df1["Annees"].max(),
                step = None,
                value = df1["Annees"].min(),
                marks={str(Annees): str(Annees) for Annees in df1['Annees'].unique()},
                    id='year-slider')
            ], style={'width': '49%', 'float': 'left', 'display': 'inline-block'}
            ),
           html.Div([
            dcc.Dropdown(
                   df1["region_name"].unique(),
                   value = "Mayotte" ,
                   id = "region"
               ),
               dcc.Graph(id ="graphe2")
            ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'}
           )
        ]),
    html.Div([
        html.Div([
            dcc.Dropdown(["Sexe","Région"],
                   value = "Sexe" ,
                   id = "SexeRegion"
               ),
            dcc.RadioItems(
                    ["Décés", "Réanimation", "Hospitalisation"],
                    value = "Décés",
                    id= "freq",
                    labelStyle={'display': 'inline-block', 'marginTop': '5px'}
                ),
               dcc.Graph(id = "graphe3")
            ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'}
           )
        ]),
    html.Div([
            dcc.RadioItems(
       options=[
           {'label': 'Régions', 'value': 'Regions'},
           {'label': 'Département', 'value': 'Departements'},
       ],
       value='Regions',
       id = "regDep"
    ) , 
     dcc.Graph(id = "graphe4") ], style={'width': '49%', 'float': 'left', 'display': 'inline-block'})
    

])

       

@app.callback(
    Output("graphe1","figure"),
    Input("year-slider", "value")
)
def update_figure1(selected):
    filter_df1 = df1[df1["Annees"]==selected]
    fig1 = px.scatter(filter_df1, x = "rea", y = "hosp", size = "dc", color = "region_name", hover_name = "dep_name", log_x=True, size_max=30 )
    
    return fig1

@app.callback(
    Output("graphe2","figure"),
    Input("region", "value")
    
)

def update_figure2(region):
    filter_df2 = df1[df1["region_name"]==region]
    dc = go.Scatter(
                    x = filter_df2["Annees"],
                    y = filter_df2["dc"],
                    mode = 'lines',
                    name = 'dc'
                   )
    rea = go.Scatter(
                    x = filter_df2["Annees"],
                    y = filter_df2["rea"],
                    mode = 'lines',
                    name = 'rea'
                   )
    hosp = go.Scatter(
                    x = filter_df2["Annees"],
                    y = filter_df2["hosp"],
                    mode = 'lines',
                    name = 'hosp'
                   )
    layout = go.Layout(
                    title = 'Covid Situation', # Title
                    xaxis = dict(title = 'Year'), # x-axis label
                    yaxis = dict(title = 'Frequences'))
    data = [dc, rea, hosp]
    fig2 = go.Figure(data=data,layout=layout)
    
    return fig2

@app.callback(
    Output("graphe3","figure"),
    Input("SexeRegion", "value"),
    Input("freq", "value")
)

def update_figure3(regionSexe,Freq):
    if regionSexe == "Sexe":
        if Freq == "Décés":
            df3 = dataframeSexe("Décés")
            data = go.Pie(
             values= df3["dc"],
             labels= df3["sexe"]
              )


            layout = go.Layout(
                   title=dict(text = "numbrer death of covid 19 in French according by sexe",x=0.46,y=0.95,font_size=20)
                  )

            fig3 = go.Figure(data=data,layout=layout)
        if Freq == "Réanimation":
            df3 = dataframeSexe("Réanimation")
            data = go.Pie(
             values= df3["rea"],
             labels= df3["sexe"]
              )


            layout = go.Layout(
                   title=dict(text = "numbrer reanimation of covid 19 in French according by sexe",x=0.46,y=0.95,font_size=20)
                  )

            fig3 = go.Figure(data=data,layout=layout)
            
        if Freq == "Hospitalisation":
            df3 = dataframeSexe("Hospitalisation")
            data = go.Pie(
             values= df3["hosp"],
             labels= df3["sexe"]
              )
            layout = go.Layout(
                   title=dict(text="numbrer Hospitalisation of covid 19 in French according by sexe", x=0.46,y=0.95,font_size=20)

            fig3 = go.Figure(data=data,layout=layout)
    else:
        if Freq == "Décés":
            df3 = dataframeRegion("Décés")
            fig3 = px.bar(df3, x = "dc", y = "region_name",
            color = "region_name", 
            orientation = "h",
            title = "numbrer deaths of covid 19 in French according by region",
                text_auto='.2s')
        if Freq == "Réanimation":
            df3 = dataframeRegion("Réanimation")
            fig3 = px.bar(df3, x = "rea", y = "region_name",
            color = "region_name", 
            orientation = "h",
            title = "numbrer reanimations of covid 19 in French according by region",
            text_auto='.2s')

        if Freq == "Hospitalisation":
            df3 = dataframeRegion("Hospitalisation")
            fig3 = px.bar(df3, x = "hosp", y = "region_name",
            color = "region_name",
            orientation = "h",
            title= "numbrer Hospitalisations of covid 19 in French according by region",
                text_auto='.2s')

    return fig3



SyntaxError: '(' was never closed (397264783.py, line 146)

SyntaxError: '(' was never closed (397264783.py, line 146)

In [ ]:
@app.callback(
    Output("graphe4","figure"),
    Input("regDep","value")
)

def update_figure(choice):
    if choice == "Regions":
        with open("regions.geojson") as f:
            region = json.loads(f.read())
        df = data.groupby("region_name", as_index=False)["dc"].sum()
        # choropleth_mapbox
        fig4 = px.choropleth_mapbox(data_frame=df,
                     geojson = region,
                     locations ="region_name",
                     color="dc",
                     featureidkey='properties.nom',
                     color_continuous_scale="teal",
                     mapbox_style="carto-positron",
                     zoom=4.0,
                     center = {"lat": 47.0, "lon": 0.0},
                     opacity=0.5,
                     labels={'Number dc ': 'Number deaths covid in French'}
                    )
    else:
        with open("departements.geojson") as f:
            departement = json.loads(f.read())
        df = data.groupby("dep_name", as_index=False)["dc"].sum()
        # choropleth_mapbox
        fig4 = px.choropleth_mapbox(data_frame=df,
                     geojson = departement,
                     locations ="dep_name",
                     color="dc",
                     featureidkey='properties.nom',
                     color_continuous_scale="teal",
                     mapbox_style="carto-positron",
                     zoom=4.0,
                     center = {"lat": 47.0, "lon": 0.0},
                     opacity=0.5,
                     labels={'Number dc ': 'Number deaths covid in French'}
                    )
     
    
    return fig4

In [ ]:
if __name__ == '__main__':
    #app.run_server(mode="inline", host="localhost",port=8051)
    app.run_server(debug = True , port=8053)